In [7]:
import torch
def _generate_negatives(z):
    """Generate negative samples to compare each sequence location against"""
    batch_size, feat, full_len = z.shape
    z_k = z.permute([0, 2, 1]).reshape(-1, feat)
    with torch.no_grad():
        # candidates = torch.arange(full_len).unsqueeze(-1).expand(-1, self.num_negatives).flatten()
        negative_inds = torch.randint(0, full_len-1, size=(batch_size, full_len * 20))
        # From wav2vec 2.0 implementation, I don't understand
        # negative_inds[negative_inds >= candidates] += 1

        for i in range(1, batch_size):
            negative_inds[i] += i * full_len

    z_k = z_k[negative_inds.view(-1)].view(batch_size, full_len, 20, feat)
    return z_k, negative_inds

In [30]:
import wandb
run = wandb.init()
artifact = run.use_artifact('sccn/eeg-ssl/model-1t2pzp1g:v7', type='model')
artifact_dir = artifact.download()

wandb: Currently logged in as: dt-young112 (sccn) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


wandb: Downloading large artifact model-1t2pzp1g:v7, 443.31MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:0.6


In [9]:
_generate_negatives(torch.randn(2, 3, 20))

(tensor([[[[ 1.1702, -1.4821, -1.6744],
           [ 0.7960, -0.8282,  1.3160],
           [-0.0633,  0.5059, -0.6821],
           ...,
           [ 1.1702, -1.4821, -1.6744],
           [ 0.4705,  1.6255, -0.0271],
           [ 0.5236,  1.1841, -1.6002]],
 
          [[-2.8905,  1.5216,  1.0312],
           [-0.1559,  0.0274,  0.2242],
           [ 0.7960, -0.8282,  1.3160],
           ...,
           [-0.1080,  0.7950,  0.6234],
           [-2.8905,  1.5216,  1.0312],
           [ 0.2810,  0.9419,  0.5902]],
 
          [[ 0.7960, -0.8282,  1.3160],
           [-1.7643,  0.4037,  0.4514],
           [-0.7473,  0.7631,  1.3274],
           ...,
           [ 0.4342, -1.3391, -1.1102],
           [ 0.2810,  0.9419,  0.5902],
           [-1.6061,  0.5028, -0.9767]],
 
          ...,
 
          [[ 0.0488,  0.9339, -0.7958],
           [-0.1559,  0.0274,  0.2242],
           [-0.1080,  0.7950,  0.6234],
           ...,
           [-0.7473,  0.7631,  1.3274],
           [ 0.0488,  0.9339, 

In [8]:
from braindecode.models import Deep4Net
model = Deep4Net(n_chans=128, n_outputs=2, n_times=500)
print(model)

Layer (type (var_name):depth-idx)             Input Shape               Output Shape              Param #                   Kernel Shape
Deep4Net (Deep4Net)                           [1, 128, 500]             [1, 2]                    --                        --
├─Ensure4d (ensuredims): 1-1                  [1, 128, 500]             [1, 128, 500, 1]          --                        --
├─Rearrange (dimshuffle): 1-2                 [1, 128, 500, 1]          [1, 1, 500, 128]          --                        --
├─CombinedConv (conv_time_spat): 1-3          [1, 1, 500, 128]          [1, 25, 491, 1]           80,275                    --
├─BatchNorm2d (bnorm): 1-4                    [1, 25, 491, 1]           [1, 25, 491, 1]           50                        --
├─ELU (conv_nonlin): 1-5                      [1, 25, 491, 1]           [1, 25, 491, 1]           --                        --
├─MaxPool2d (pool): 1-6                       [1, 25, 491, 1]           [1, 25, 163, 1]           -- 

In [16]:
import torch
mask = (torch.rand((20,10)) < 0.5).bool() 
print(mask)
def get_nonzero_indices_batch(input_tensor):
  """
  Gets the indices of False elements along the batch dimension (dimension 0).

  Args:
    input_tensor: A PyTorch tensor.

  Returns:
    A list of LongTensors, where each LongTensor contains the indices
    along the batch dimension where nonzero elements exist for the
    corresponding element in the remaining dimensions.
  """
  nonzero_indices_list = []
  # Iterate through each element along the batch dimension
  for i in range(input_tensor.size(0)):
    nonzero_indices = torch.nonzero(~input_tensor[i])[:, 0]
    nonzero_indices_list.append(nonzero_indices)
  return nonzero_indices_list

non_masked_inds = get_nonzero_indices_batch(mask)
print(non_masked_inds)
print(torch.randint(0, non_masked_inds[0].shape[0], (5,)))
non_masked_inds[0][torch.randint(0, non_masked_inds[0].shape[0], (5,))]

tensor([[ True,  True, False, False,  True, False, False,  True, False, False],
        [ True,  True, False,  True, False,  True,  True,  True,  True,  True],
        [False,  True, False, False,  True,  True,  True, False, False,  True],
        [ True,  True, False, False,  True,  True, False, False,  True,  True],
        [ True,  True,  True, False, False, False, False,  True, False, False],
        [ True, False,  True, False,  True,  True,  True, False,  True,  True],
        [ True, False,  True,  True, False, False,  True,  True,  True,  True],
        [False, False,  True,  True,  True, False,  True, False, False, False],
        [ True,  True, False,  True,  True,  True, False, False,  True,  True],
        [ True, False, False, False,  True, False, False,  True, False,  True],
        [ True, False, False, False,  True,  True, False,  True, False,  True],
        [False,  True, False, False,  True,  True,  True,  True,  True, False],
        [False, False,  True, False,  Tr

tensor([8, 9, 3, 3, 2])

In [ ]:
def _generate_negatives_from_sequence(z, mask_t, num_negatives=5):
        """Generate negative samples from the encoder output of the same sequence but different time steps"""
        batch_size, feat, full_len = z.shape
        assert mask_t.shape == (batch_size, full_len), f"mask_t {mask_t.shape} should be (B, seq_len)"

        with torch.no_grad():
            # gather False indices in mask_t for each batch item
            non_masked_inds = [torch.nonzero(~item)[:, 0] for item in mask_t]
            # for each batch item, sample with replacement self.num_negatives samples from negative_inds
            negatives_inds = []
            for i in range(batch_size):
                negatives_inds.append(non_masked_inds[i][torch.randint(0, non_masked_inds[i].shape[0], (num_negatives,))])
            negatives_inds = torch.stack(negatives_inds, dim=0) # (B, num_negatives)
            assert negatives_inds.shape == (batch_size, num_negatives), f"negatives indices {negatives_inds.shape} should be (B, num_negatives)"
            assert ~torch.any(torch.gather(mask, dim=1, index=negatives_inds)), f"mask[negative_inds] should be False"
            
            z_k = z.clone().permute([0, 2, 1]) # (B, seq_len, F)
            negatives = torch.gather(z_k, dim=1, index=negatives_inds.unsqueeze(-1).expand(-1, -1, feat)) # (B, num_negatives, F)
            negatives = negatives.permute([0, 2, 1]) # (B, F, num_negatives)
            assert negatives.shape == (batch_size, feat, num_negatives), f"negatives {negatives.shape} should be (B, F, num_negatives)"

            return negatives, negatives_inds

In [29]:
_, _ = _generate_negatives_from_sequence(torch.randn((20,10,10)), mask)

In [5]:
# Example of using it in your original code.
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

class SimpleNet(nn.Module):
    def __init__(self, input_dim):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(input_dim, 100)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(100, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

#  Fix: Ensure x and labels are defined.
input_dim = 10
batch_size = 32
x = torch.randn(batch_size, input_dim) #  <---  No requires_grad here initially
labels = torch.randn(batch_size, 1, requires_grad=False)

# Set requires_grad to True AFTER x is created.
# x.requires_grad_(True)

projection_head = SimpleNet(input_dim).to(x.device)
print('projection_head', list(projection_head.parameters()))
projection_head.train()
optimizer = optim.Adam(projection_head.parameters(), lr=0.001)
print(projection_head)

for e in range(100):
    optimizer.zero_grad()
    preds = projection_head(x)
    print('preds', preds.shape)
    print('labels', labels.shape)
    loss = F.mse_loss(preds, labels)
    print('loss', loss)
    loss.backward()
    optimizer.step()
print("Training Complete")


projection_head [Parameter containing:
tensor([[ 0.1709,  0.2581, -0.3148,  0.1944, -0.2157, -0.1433,  0.2040, -0.0190,
          0.2481, -0.0744],
        [ 0.1580,  0.2435, -0.1837, -0.0369, -0.3125,  0.1737, -0.1756,  0.0784,
         -0.2865, -0.2332],
        [-0.2255,  0.0651,  0.2813,  0.1326, -0.0558,  0.3159, -0.1382, -0.1883,
         -0.1853, -0.0718],
        [ 0.0339, -0.0989, -0.1561, -0.1612, -0.0054,  0.2111, -0.1528, -0.0160,
          0.1757, -0.3142],
        [ 0.2914, -0.0747,  0.0996,  0.0055, -0.0389, -0.1731, -0.1679, -0.2122,
          0.2429, -0.3097],
        [ 0.0695, -0.2420, -0.0758,  0.0221, -0.2116,  0.1836,  0.3137, -0.0517,
         -0.1626, -0.1631],
        [-0.0165,  0.1339, -0.1492,  0.2247, -0.2080,  0.2238, -0.1984,  0.1170,
          0.2981,  0.0101],
        [-0.1110, -0.1754,  0.0034, -0.2797, -0.0463,  0.1407, -0.2960,  0.0853,
          0.2184,  0.1719],
        [ 0.1851, -0.1137,  0.0795, -0.2013, -0.0048, -0.3054,  0.2835, -0.1286,
        